In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

print("--- ETAPA 2: ENGENHARIA DE FEATURES (LENDO E SALVANDO JSON) ---")

# Bloco 1: Carregar os dados consolidados do JSON
print("\nCarregando 'dados_consolidados.json'...")
try:
    # Usamos lines=True porque salvamos nesse formato
    df = pd.read_json('dados_consolidados.json', lines=True)
    print("Dados carregados com sucesso.")
except FileNotFoundError:
    print("Erro: O arquivo 'dados_consolidados.json' não foi encontrado. Execute a Etapa 1 primeiro.")
    exit()

# Bloco 2: Limpeza e Combinação de Campos de Texto
print("\nLimpando e combinando campos de texto...")
cols_vaga = ['info.titulo_vaga', 'perfil.principais_atividades', 'perfil.competencia_tecnicas_e_comportamentais']
cols_candidato = ['cv_pt', 'informacoes_profissionais.conhecimentos_tecnicos', 'informacoes_profissionais.area_atuacao', 'informacoes_profissionais.qualificacoes']
for col in cols_vaga + cols_candidato:
    df[col] = df[col].fillna('')
df['texto_vaga'] = df[cols_vaga].agg(' '.join, axis=1)
df['texto_candidato'] = df[cols_candidato].agg(' '.join, axis=1)
print("Campos de texto combinados.")

# Bloco 3: Feature de Similaridade de Texto (NLP)
print("\nCalculando a similaridade de texto com TF-IDF...")
vectorizer = TfidfVectorizer(stop_words='english')
similarities = []
for index, row in df.iterrows():
    try:
        tfidf_matrix = vectorizer.fit_transform([row['texto_vaga'], row['texto_candidato']])
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        similarities.append(similarity)
    except ValueError:
        similarities.append(0)
df['similaridade_texto'] = similarities
print("Feature 'similaridade_texto' criada.")

# Bloco 4: Features de Compatibilidade de Nível de Idioma
print("\nCriando features de compatibilidade de idioma...")
mapa_niveis = {'Nenhum': 0, 'Básico': 1, 'Intermediário': 2, 'Avançado': 3, 'Fluente': 4}
df['vaga_nivel_ingles_num'] = df['perfil.nivel_ingles'].apply(lambda x: mapa_niveis.get(x, 0))
df['candidato_nivel_ingles_num'] = df['formacao_e_idiomas.nivel_ingles'].apply(lambda x: mapa_niveis.get(x, 0))
df['match_nivel_ingles'] = (df['candidato_nivel_ingles_num'] >= df['vaga_nivel_ingles_num']).astype(int)
print("Feature 'match_nivel_ingles' criada.")

# Bloco 5: Feature de Compatibilidade SAP
print("\nCriando feature de compatibilidade SAP...")
df['vaga_requer_sap'] = (df['info.vaga_sap'] == 'Sim').astype(int)
df['candidato_tem_sap'] = df['texto_candidato'].str.contains('sap', flags=re.IGNORECASE, regex=True).astype(int)
df['match_sap'] = ((df['vaga_requer_sap'] == 0) | ((df['vaga_requer_sap'] == 1) & (df['candidato_tem_sap'] == 1))).astype(int)
print("Feature 'match_sap' criada.")

# Bloco 6: Seleção e Salvamento das Features Finais em JSON
print("\nSelecionando features finais e salvando o resultado...")
features_selecionadas = [
    'id_vaga',
    'prospect.codigo', # id_candidato
    'similaridade_texto',
    'match_nivel_ingles',
    'match_sap',
    'match' # Nossa variável-alvo
]
df_features = df[features_selecionadas]

# Salva o DataFrame para a próxima etapa em formato JSON
df_features.to_json('dados_com_features.json', orient='records', lines=True, force_ascii=False)
print("\n--- DataFrame com Novas Features ---")
print(df_features.head())
print(f"\nDimensões do DataFrame de features: {df_features.shape}")
print("\nArquivo 'dados_com_features.json' salvo com sucesso!")
print("\n--- ETAPA 2 CONCLUÍDA ---")

--- ETAPA 2: ENGENHARIA DE FEATURES (LENDO E SALVANDO JSON) ---

Carregando 'dados_consolidados.json'...
Dados carregados com sucesso.

Limpando e combinando campos de texto...
Campos de texto combinados.

Calculando a similaridade de texto com TF-IDF...
Feature 'similaridade_texto' criada.

Criando features de compatibilidade de idioma...
Feature 'match_nivel_ingles' criada.

Criando feature de compatibilidade SAP...
Feature 'match_sap' criada.

Selecionando features finais e salvando o resultado...

--- DataFrame com Novas Features ---
   id_vaga  prospect.codigo  similaridade_texto  match_nivel_ingles  \
0     4530            25632            0.260992                   1   
1     4530            25529            0.132114                   1   
2     4531            25364            0.112634                   1   
3     4531            25360            0.085489                   1   
4     4533            26338            0.169030                   1   

   match_sap  match  
0      